In [86]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime, timedelta
import pytz

In [87]:
API_KEY = "39357f117056c535298fe0df516ce3e3"
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

In [88]:
def weather(city):
    url = f"{BASE_URL}?q={city}&appid={API_KEY}&units=metric"
    try:
        response = requests.get(url)
        data = response.json()

        if response.status_code != 200:
            raise Exception(f"Error: {data.get('message', 'Unknown error')}")

        return {
            'city': data.get('name', 'Unknown'),
            'current_temp': round(data['main'].get('temp', 0)),
            'feels_like': round(data['main'].get('feels_like', 0)),
            'temp_min': round(data['main'].get('temp_min', 0)),
            'temp_max': round(data['main'].get('temp_max', 0)),
            'humidity': round(data['main'].get('humidity', 0)),
            'description': data['weather'][0]['description'] if 'weather' in data else 'N/A',
            'country': data['sys'].get('country', 'N/A'),
            'WindGustSpeed': data['wind'].get('speed', 0),
            'WindGustDir': data['wind'].get('deg', 0),
            'Pressure': data['main'].get('pressure', 0)
        }
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return {}

In [89]:
def load_data(filename):
    try:
        df = pd.read_csv(filename, sep=',', encoding='utf-8')
        df = df.dropna()
        df = df.drop_duplicates()
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return pd.DataFrame()

In [90]:
def prepare(data):
    le = LabelEncoder()
    if 'WindGustDir' in data.columns:
        data['WindGustDir'] = le.fit_transform(data['WindGustDir'].astype(str))
    if 'RainTomorrow' in data.columns:
        data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'].astype(str))
    required_columns = ['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure']
    available_columns = [col for col in required_columns if col in data.columns]
    if len(available_columns) < len(required_columns):
        print(f"Warning: Missing columns {set(required_columns) - set(available_columns)}")
    X = data[available_columns]
    y = data['RainTomorrow'] if 'RainTomorrow' in data.columns else None
    return X, y, le

In [91]:
def train_rain_model(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100 ,random_state=42)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("Mean Squared Error:")
    print(mean_squared_error(y_test, y_pred))
    return model

In [92]:
def prepare_regression_data(data, feature):
    x, y = [], []
    for i in range(len(data) - 1):
        x.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i + 1])
    x = np.array(x).reshape(-1, 1)
    y = np.array(y)
    return x, y

In [93]:
def train_regression_model(x, y):
    model = RandomForestRegressor(random_state=42)
    model.fit(x, y)
    return model

In [94]:
def predict_future(model, value):
    prediction = [value]
    for i in range(5):
        next_value = model.predict(np.array([prediction[-1]]).reshape(-1, 1))
        prediction.append(next_value[0])
    return prediction[1:]

In [122]:
def weather_view():
    city = input("Enter city name: ")

    current_weather = weather(city)
    if not current_weather:
        print("Failed to retrieve weather data.")
        return

    historical_data = load_data('/content/weather.csv')

    x, y, le = prepare(historical_data)
    rain_model = train_rain_model(x, y)

    wind_deg = current_weather['WindGustDir'] % 360
    compass_points = [
        ("N", 0, 22.5), ("NE", 22.5, 67.5), ("E", 67.5, 112.5), ("SE", 112.5, 157.5),
        ("S", 157.5, 202.5), ("SW", 202.5, 247.5), ("W", 247.5, 292.5), ("NW", 292.5, 337.5), ("N", 337.5, 360)
    ]
    compass_direction = next((point for point, start, end in compass_points if start <= wind_deg < end), "Unknown")
    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    current_data = {
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'],
        'WindGustDir': compass_direction_encoded,
        'WindGustSpeed': current_weather['WindGustSpeed'],
        'Humidity': current_weather['humidity'],
        'Pressure': current_weather['Pressure'],
    }

    current_df = pd.DataFrame([current_data])
    rain_prediction = rain_model.predict(current_df)[0]
    rain_probability = "Yes" if rain_prediction == 1 else "No"

    x_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
    x_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')
    x_wind, y_wind = prepare_regression_data(historical_data, 'WindGustSpeed')

    temp_model = train_regression_model(x_temp, y_temp)
    hum_model = train_regression_model(x_hum, y_hum)
    wind_model = train_regression_model(x_wind, y_wind)

    future_temps = predict_future(temp_model, current_weather['temp_min'])
    future_hums = predict_future(hum_model, current_weather['humidity'])
    future_winds = predict_future(wind_model, current_weather['WindGustSpeed'])

    timezone = pytz.timezone("Africa/Cairo")
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:%M") for i in range(5)]

    future_rain = future_rain_prediction(rain_model, current_data, future_times=future_times)

    print(f"City: {current_weather['city']}, {current_weather['country']}")
    print(f"Current Temperature: {current_weather['current_temp']}°C")
    print(f"Feels Like: {current_weather['feels_like']}°C")
    print(f"Min Temperature: {current_weather['temp_min']}°C, Max Temperature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Description: {current_weather['description']}")
    print(f"Pressure: {current_weather['Pressure']} hPa")
    print(f"Wind: {current_weather['WindGustSpeed']} km/h, Direction: {compass_direction}")
    print(f"Rain Prediction: {rain_probability}")
    print("\nFuture Weather Predictions:")
    print("Time\t\tTemp (°C)\tHumidity (%)\tWind (km/h)\tRain")
    for time, temp, hum, wind, rain in zip(future_times, future_temps, future_hums, future_winds, future_rain):
        print(f"{time}\t\t{round(temp,1)}°C\t\t{round(hum,1)}%\t\t{round(wind,1)} km/h\t{rain}")


In [124]:
weather_view()

Enter city name: beni suef
Mean Squared Error:
0.1506849315068493
City: Beni Suef, EG
Current Temperature: 12°C
Feels Like: 10°C
Min Temperature: 12°C, Max Temperature: 12°C
Humidity: 39%
Description: clear sky
Pressure: 1019 hPa
Wind: 0.12 km/h, Direction: E
Rain Prediction: No

Future Weather Predictions:
Time		Temp (°C)	Humidity (%)	Wind (km/h)	Rain
08:00		12.8°C		31.7%		28.3 km/h	No
09:00		12.4°C		38.0%		35.4 km/h	No
10:00		14.4°C		52.3%		36.1 km/h	No
11:00		13.4°C		43.4%		36.1 km/h	No
12:00		13.6°C		46.8%		36.1 km/h	No
